In [3]:
!mkdir NBMapReduce

In [4]:
!ls

MapReduce MeanMinMaksJava.ipynb
NBMapReduce


In [5]:
%%file ./NBMapReduce/NBCDriver.java
import java.io.IOException;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class NBCDriver {
	public static void main(String[] args) throws IOException, InterruptedException, ClassNotFoundException {
		Configuration conf=new Configuration();
		// The test input for which you want to find the acitivity that the Person should be doing
		conf.set("test_input", args[0]);
		Job job = new Job(conf);
		job.setJarByClass(NBCDriver.class);
		job.setJobName("Naive_Bayes_calssifier using Hadoop");
		FileInputFormat.setInputPaths(job, new Path(args[1]));
		FileOutputFormat.setOutputPath(job, new Path(args[2]));
		job.setMapperClass(NBCMap.class);
		job.setReducerClass(NBCReduce.class);
		job.setMapOutputKeyClass(IntWritable.class);
		job.setMapOutputValueClass(Text.class);
		job.setOutputKeyClass(IntWritable.class);
		job.setOutputValueClass(Text.class);
		boolean success = job.waitForCompletion(true);
		System.exit(success ? 0 : 1);
	}
}

Writing ./NBMapReduce/NBCDriver.java


In [6]:
%%file ./NBMapReduce/NBCMap.java
import java.io.IOException;
import java.util.HashMap;
import java.util.Map.Entry;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

public class NBCMap extends Mapper<LongWritable, Text, IntWritable, Text>{
	public static String output_key;
	public static String[] test_input=null;
	public static int count=0;
	public static HashMap<String,Integer> inputs=new HashMap<String,Integer>();
	public static double output_value=Double.NEGATIVE_INFINITY;
	public static HashMap<String,Double> output= new HashMap<String,Double>();
	public static HashMap<String,Double> outcome_count= new HashMap<String,Double>();
	public static HashMap<String,Double> features_count= new HashMap<String,Double>();
	public void map(LongWritable key, Text value, Context context) throws IOException, InterruptedException {
		if(test_input==null)
			test_input=context.getConfiguration().get("test_input").split("\\,");
		String[] input=value.toString().split("\\,");
		for(int j=0;j<input.length;j++){
			if(j==input.length-1){
				if(outcome_count.containsKey(input[j]))
					outcome_count.put(input[j], outcome_count.get(input[j])+1);
				else
					outcome_count.put(input[j], (double) 1);
			}
			else{
				if(input[j].contentEquals(test_input[j])){
					if(!inputs.containsKey(j+","+input[j]))
						inputs.put(j+","+input[j], 0);
					if(features_count.containsKey(j+","+input[j]+"|"+input[input.length-1]))
						features_count.put(j+","+input[j]+"|"+input[input.length-1], features_count.get(j+","+input[j]+"|"+input[input.length-1])+1);
					else
						features_count.put(j+","+input[j]+"|"+input[input.length-1], (double) 1);
				}
			}
		}
		++count;
	}
	public void cleanup(Context context) throws IOException, InterruptedException{
		
		for(Entry<String,Double> o_c:outcome_count.entrySet()){
			String output_class=o_c.getKey();
			for(Entry<String,Integer> i:inputs.entrySet()){
				if(!features_count.containsKey(i.getKey()+"|"+output_class))
					features_count.put(i.getKey()+"|"+output_class, (double) 0);
			}
			double output_class_count=o_c.getValue();
			double probability=output_class_count/count;
			for(Entry<String,Double> f_c:features_count.entrySet()){
				if(f_c.getKey().split("\\|")[1].contentEquals(output_class))
					probability=probability*(f_c.getValue()/output_class_count);
			}
			output.put(output_class, probability);
		}
		for(Entry<String,Double> o:output.entrySet()){
			if(o.getValue()>output_value){
				output_value=o.getValue();
				output_key=o.getKey();
			}
		}
		context.write(new IntWritable(1),new Text(output_key));
	}
}


Writing ./NBMapReduce/NBCMap.java


In [7]:
%%file ./NBMapReduce/NBCReduce.java
import java.io.IOException;
import java.util.HashMap;
import java.util.Map.Entry;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;

public class NBCReduce extends Reducer<IntWritable, Text, IntWritable, Text>{
	public void reduce(IntWritable key, Iterable<Text> values, Context context) throws IOException, InterruptedException{
		Double out_value=Double.NEGATIVE_INFINITY;
		String out_key=null;
		HashMap<String,Integer> final_output=new HashMap<String,Integer>();
		for(Text value:values){
			if(final_output.containsKey(value.toString()))
				final_output.put(value.toString(), final_output.get(value.toString())+1);
			else
				final_output.put(value.toString(), 1);
		}
		for(Entry<String,Integer> output:final_output.entrySet()){
			if(output.getValue()>out_value){
				out_value=(double) output.getValue();
				out_key=output.getKey();
			}
		}
		context.write(null, new Text(out_key));
	}
}


Writing ./NBMapReduce/NBCReduce.java


In [8]:
#set permission dari file *.java agar dapat dieksekusi
!chmod u+x ./NBMapReduce/*.java

In [4]:
!echo %HADOOP_HOME%

C:\opt\hadoop-2.7.3


In [1]:
import os
os.chdir("./NBMapReduce")

In [2]:
!pwd

C:\Users\Imacho\akodingDTSbatch2\koding MapReduceJava\NBMapReduce


In [7]:
!ls

NBCDriver.class
NBCDriver.java
NBCMap.class
NBCMap.java
NBCReduce.class
NBCReduce.java


In [8]:
!rm *.class

In [9]:
!ls

NBCDriver.java
NBCMap.java
NBCReduce.java


In [10]:
#jika compile di Win OS
!javac -cp "%HADOOP_HOME%\share\hadoop\common\*;%HADOOP_HOME%\share\hadoop\mapreduce\*" *.java

#jika compile di Linux OS
#!javac -cp "${HADOOP_HOME}/share/hadoop/common/*:${HADOOP_HOME}/share/hadoop/mapreduce/*" *.java

Note: NBCDriver.java uses or overrides a deprecated API.
Note: Recompile with -Xlint:deprecation for details.


In [11]:
!ls

NBCDriver.class
NBCDriver.java
NBCMap.class
NBCMap.java
NBCReduce.class
NBCReduce.java


In [24]:
!jar cvf NBMapReduce.jar *.class

added manifest
adding: NBCDriver.class(in = 1677) (out= 905)(deflated 46%)
adding: NBCMap.class(in = 4556) (out= 1909)(deflated 58%)
adding: NBCReduce.class(in = 2485) (out= 1099)(deflated 55%)


In [25]:
!ls

dataset.txt
NBCDriver.class
NBCDriver.java
NBCMap.class
NBCMap.java
NBCReduce.class
NBCReduce.java
NBMapReduce - Copy.jar
NBMapReduce.jar
output


In [61]:
!hdfs dfs -ls .

Found 3 items
drwxr-xr-x   - hue    supergroup          0 2019-09-04 03:20 input
drwxr-xr-x   - hue    supergroup          0 2019-09-04 03:24 output
drwxr-xr-x   - ImamCS supergroup          0 2019-10-10 23:47 user


In [16]:
%%file dataset.txt
Urgent,Yes,Yes,Party
Urgent,No,Yes,Study
Near,Yes,Yes,Party
None,Yes,No,Party
None,No,Yes,Pub
None,Yes,No,Party
Near,No,No,Study
Near,No,Yes,TV
Near,Yes,Yes,Party
Urgent,No,No,Study

Writing dataset.txt


In [18]:
!ls

dataset.txt
NBCDriver.class
NBCDriver.java
NBCMap.class
NBCMap.java
NBCReduce.class
NBCReduce.java
NBMapReduce.jar
output


In [52]:
!hdfs dfs -mkdir ./user

In [53]:
!hdfs dfs -mkdir ./user/ImamCS

In [54]:
!hdfs dfs -mkdir ./user/ImamCS/nb

In [55]:
!hdfs dfs -mkdir ./user/ImamCS/nb/input

In [57]:
!hdfs dfs -copyFromLocal ./dataset.txt ./user/ImamCS/nb/input

In [59]:
!hdfs dfs -ls ./user/ImamCS/nb/input

Found 1 items
-rw-r--r--   1 ImamCS supergroup        192 2019-10-10 23:49 user/ImamCS/nb/input/dataset.txt


In [68]:
!hdfs dfs -cat ./user/ImamCS/nb/input/dataset.txt

Urgent,Yes,Yes,Party
Urgent,No,Yes,Study
Near,Yes,Yes,Party
None,Yes,No,Party
None,No,Yes,Pub
None,Yes,No,Party
Near,No,No,Study
Near,No,Yes,TV
Near,Yes,Yes,Party
Urgent,No,No,Study


In [4]:
!%HADOOP_HOME%/bin/hadoop jar NBMapReduce.jar NBCDriver "Urgent,No,No" \
"./user/ImamCS/nb/input/dataset.txt" \
"./user/ImamCS/nb/output2"

19/10/11 01:37:33 INFO client.RMProxy: Connecting to ResourceManager at localhost/127.0.0.1:8050
19/10/11 01:37:34 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
19/10/11 01:37:36 INFO input.FileInputFormat: Total input paths to process : 1
19/10/11 01:37:36 INFO mapreduce.JobSubmitter: number of splits:1
19/10/11 01:37:37 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1570732148023_0003
19/10/11 01:37:37 INFO impl.YarnClientImpl: Submitted application application_1570732148023_0003
19/10/11 01:37:38 INFO mapreduce.Job: The url to track the job: http://XVisible-Teams:8088/proxy/application_1570732148023_0003/
19/10/11 01:37:38 INFO mapreduce.Job: Running job: job_1570732148023_0003
19/10/11 01:38:15 INFO mapreduce.Job: Job job_1570732148023_0003 running in uber mode : false
19/10/11 01:38:15 INFO mapreduce.Job:  map 0% reduce 0%
19/10/11 01:38:34

In [5]:
!hadoop fs -cat ./user/ImamCS/nb/output2/*

Study


![title](img/thumbs-up.png)